## CMS document

### HIN 2021 tracking selection development
The version of cmssw can be checked by
```bash
scram list
```
Setupt the workspace
```bash
scram p -n hi2021_runPrep_11_1_0_pre6 CMSSW_11_1_0_pre6
cd hi2021_runPrep_11_1_0_pre6/src
cmsenv
scram b -j4
```

### Generating the sample
The workflow: Make minbias sample (Hydjet) + embedding signal (PYTHIA) + HLT simulation + reco

#### Minbias sample

We first need to generate the minbias samples as background and embed the QCD jet sample into it.

For generating the minbias sample, the command is
```bash
cmsDriver.py Hydjet_Quenched_B12_5020GeV_cfi  --conditions auto:phase1_2021_realistic_hi -n 1 --era Run3_pp_on_PbPb --eventcontent RAWSIM --relval 2000,1 -s GEN,SIM --datatier GEN-SIM --beamspot RealisticPbPbCollision2018 --geometry DB:Extended
```
where the beamspot used here is "RealisticPbPbCollision2018" and need to turn the `cFlag` on after generated the cfg file manually (otherwise, it will only generate the sample with b in the range specified in cfg).

#### Embedding signal samples
To generate the QCD jets with Pthat 80-120 (`QCD_Pt_80_120_14TeV_TuneCP5_cfi`) samples for embedding (using the selfgenerated sample for embedding):
```bash
cmsDriver.py QCD_Pt_80_120_14TeV_TuneCP5_cfi --mc --pileup HiMixGEN --pileup_input "dbs:/MinBias/wangx-minBias_HydjetDrum5F_GEN_SIM_PU-43543e2e0073ceac48fd2078951ca5eb/USER instance=prod/phys03" --eventcontent RAWSIM --datatier GEN-SIM --conditions auto:phase1_2021_realistic_hi --beamspot MatchHI --step GEN,SIM --scenario HeavyIons --geometry DB:Extended --era Run3_pp_on_PbPb --no_exec
```

#### Reconstruction
The embedding PYTHIA into HYDJET + simulation configure can be generated as
```bash
cmsDriver.py  step1 --mc --pileup HiMix --pileup_input dbs:/MinBias/wangx-minBias_HydjetDrum5F_GEN_SIM_PU-43543e2e0073ceac48fd2078951ca5eb/USER instance=prod/phys03 --eventcontent FEVTDEBUGHLT --datatier GEN-SIM-DIGI-RAW-HLTDEBUG --conditions auto:phase1_2021_realistic_hi --step DIGI:pdigi_hi,L1,DIGI2RAW,HLT:@fake2 --scenario HeavyIons --geometry DB:Extended --era Run3_pp_on_PbPb -n 2 --no_exec
```

The reconsctruction configuration is 
```bash
cmsDriver.py step3  --runUnscheduled  --conditions auto:phase1_2021_realistic_hi -s RAW2DIGI,L1Reco,RECO --datatier GEN-SIM-RECODEBUG -n 2 --era Run3_pp_on_PbPb --eventcontent RECODEBUG --no_exec
```
In this example, a RECODEBUG format file will produced after the reco. 


|type|stats|event content| PD name|
|---|---|---|---|
|minbias|10k|GEN | `/MinBias/wangx-minBias_HydjetDrum5F_GEN_SIM_PU-43543e2e0073ceac48fd2078951ca5eb/USER instance=prod/phys03`|
|Embedded QCD jet [80,120]|10k|GEN|`/EmbeddedSample/wangx-QCD_Pt_80_120_embedding_HydjetDrum5F_GEN_SIM_PU-4d98a1cfa9b29ee83ac89ca089c090ad/USER instance=prod/phys03`|
|Embedded QCD jet [80,120]|10k|FEVTDEBUGHLT|`/EmbeddedSample/wangx-embeddingSample_QCD_Pthat_80_120_5TeV_TuneCP5_HydjetDrum5F_GEN_SIM_PU_11_2_0_pre8_Condition-c857ea772e9e4b6d4786f4b9b3904718/USER instance=prod/phys03`|

## Tracking selection
Setup the tracking code in the subdirectory under the `$CMSSW_BASE/src/`
```bash
git clone git@github.com:Taburis/MVATracking.git
```
A modification is needed for the file of `runTheNtuple.py` since unexpected changes made from the CMSSW updating. A quick way to do so is to comparing the make ntuple cfg with the RECO cfg at that version generated by
```bash
cmsDriver.py step3  --runUnscheduled  --conditions auto:phase1_2021_realistic_hi -s RAW2DIGI,L1Reco,RECO --datatier GEN-SIM-RECODEBUG -n 2 --era Run3_pp_on_PbPb --eventcontent RECODEBUG --no_exec
```

The `TMVA` are not supported by the GForest anymore, some modification are needed.

## Cut based selector
An modifided MVA classifier has been used to implemented the [cut-based selections](
https://github.com/cms-sw/cmssw/blob/CMSSW_10_3_X/RecoTracker/FinalTrackSelectors/plugins/TrackCutClassifier.cc).
The usage is similar like the [TrackMVAClassifier](https://github.com/cms-sw/cmssw/blob/CMSSW_10_3_X/RecoTracker/FinalTrackSelectors/plugins/DefaultTrackMVAClassifier.cc), by setting the cfi:
```py
process.moduleName = cms.EDProducer("TrackCutClassifier",
    beamspot = cms.InputTag("offlineBeamSpot"),
    ignoreVertices = cms.bool(False),
    mightGet = cms.optional.untracked.vstring,
    mva = cms.PSet(
        #cuts listed here, for example:
        maxChi2 = cms.vdouble(9999.0, 9999.0, 9999.0),
        maxChi2n = cms.vdouble(10.0, 1.0, 0.4)
        ),
     # depends on the three numbers listed above
    qualityCuts = cms.vdouble((-0.7, 0.1, 0.7),
    src = cms.InputTag("inputCandidateName"),
    vertices = cms.InputTag("firstStepPrimaryVertices")
)

```

In [ ]:
'HBHENoiseFilterResultRun2Loose',
                'pprimaryVertexFilter',
                'collisionEventSelectionAODv2',
                'phfCoincFilter2Th4',
                'pclusterCompatibilityFilter'